In [11]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
from visualisation_tools import *
from model_generator import *
from qsq_protocol import *
from csv_utils import *
from optimizer_utils import *
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "text.usetex": False, # Remove this line if your code doesn't run/ take too long
    "font.family": 'Times New Roman',
    "figure.dpi": 100,
    "font.size": 14
})
import os
import csv

In [12]:
#  Define the CSV file path
csv_file = 'data2.csv'
# generate_model_data(csv_file, 'uniform',2000)
# for strength in np.linspace(0.3,0.5,100):
#     generate_model_data(csv_file, 'perturbed',30,perturbation=strength)
generate_model_data(csv_file, 'random',10)

# Usage
data = load_model_data(csv_file)

# # Access the data by variable name, for example:
# print(data['models'])     # Access the models (now Qobjs)



In [13]:
fill_model_gauge_fidelity("data2.csv",10)

20 rows to be processed
[10/20] flushed (50.0%)


KeyboardInterrupt: 

In [14]:
from csv_utils import _rewrite_csv

In [15]:
def generate_gauge_models(input_csv, output_csv):
    import csv
    import datetime
    from qutip import Qobj

    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

    header = [
        'model_id',
        'model',
        'P_vector',
        'model_average_fidelity',
        'model_average_fidelity_gauge'
    ]

    with open(input_csv, mode='r', newline='') as infile:
        reader = csv.DictReader(infile)

        for i, row in enumerate(reader):
            # Reconstruct original model
            model_data = eval(row['model'])
            rho, gate, M = [Qobj(m) for m in model_data]

            # --- Compute new gauge model ---
            U_gauge = rand_unitary(2)
            rho_g = U_gauge @ rho @ U_gauge.dag()
            gate_g = U_gauge @ gate @ U_gauge.dag()
            M_g = U_gauge @ M @ U_gauge.dag()

            # Convert new model to list format
            new_model = [
                rho_g.full().tolist(),
                gate_g.full().tolist(),
                M_g.full().tolist()
            ]

            # Compute new P_vector
            new_P_vector = experiment_prob_failing(rho_g, gate_g, M_g)

            # Build new model_id
            model_id = f"{timestamp}_gauge_{i}"

            # Keep original fidelities
            model_avg_fid = float(row['model_average_fidelity'])
            model_avg_fid_gauge = row.get('model_average_fidelity_gauge', '')

            new_row = [
                model_id,
                new_model,
                new_P_vector,
                model_avg_fid,
                model_avg_fid_gauge
            ]

            write_to_csv(output_csv, header, new_row)


In [17]:
generate_gauge_models("data.csv", "data3.csv")